In [1]:
from proj import *
import numpy as np

In [6]:
xs = getXS()

In [10]:
xs[1].sigA[1]

0.0203

In [5]:
xs[0].sigTr[0]

0.223775

In [2]:
solnMesh = Mesh(10, 
                np.linspace(0.0,400,11), 
                [2,1,1,1,3,3,1,1,1,2],
                [2,1])

In [5]:
solnMesh.mat

[2, 1, 1, 1, 3, 3, 1, 1, 1, 2]

In [15]:
for i in range(solnMesh.nX) :
    print xs[solnMesh.mat[i]-1].sigA[1]

0.0203
0.158201
0.158201
0.158201
0.182224
0.182224
0.158201
0.158201
0.158201
0.0203
